In [60]:
import pandas as pd

# Exercise 1

In [61]:
# This code is provided for you. Please read it and understand what it's doing
def payment_df_to_cohort_df(payment_df):
    # Calculate the cohort month for each customer. This is the first month that the customer made a payment
    cohort_df = payment_df.groupby('customer_id').payment_date.min().reset_index()
    cohort_df['cohort'] = cohort_df['payment_date'].dt.to_period('M').dt.to_timestamp()
    payment_df = payment_df.merge(cohort_df[['customer_id', 'cohort']], on='customer_id')

    # Round payments to the nearest month. month needs to be a pd.datetime still
    # Using payment_month simplifies how CVF thinks about cohorts
    payment_df['payment_month'] = payment_df['payment_date'].dt.to_period('M').dt.to_timestamp()

    # Calculate the payment period. However, I want the result to be an integer month
    # payment_period integers are one way to understand cohorts
    payment_df['payment_period'] = (
        (payment_df['payment_month'].dt.year - payment_df['cohort'].dt.year) * 12 +
        (payment_df['payment_month'].dt.month - payment_df['cohort'].dt.month)
    )

    grouped_df = payment_df.groupby(["cohort", "payment_period"])["amount"].sum()
    grouped_df = grouped_df.reset_index().pivot(index="cohort", columns="payment_period", values="amount").rename_axis(index=None, columns=None)
    return grouped_df




In [62]:
test_payment_df = pd.DataFrame({
    'customer_id': [1, 1, 2, 2, 3],
    'payment_date': pd.to_datetime(['2020-01-01', '2020-02-01', '2020-02-01', '2020-03-01', '2020-04-01']),
    'amount': [200, 100, 400, 300, 500]
})


In [63]:
EXERCISE_1_COHORT_DF = payment_df_to_cohort_df(test_payment_df)
EXERCISE_1_COHORT_DF

,0,1
2020-01-01,200.0,100.0
2020-02-01,400.0,300.0
2020-04-01,500.0,NaN


In [64]:
# Example spend file. Each cohort is spending $1000 a month
# We don't actually use the spend_df until exercise 3
EXERCISE_SPEND_DF = pd.DataFrame({
    "cohort_start_at": [pd.Timestamp(date) for date in ['2020-01-01 00:00:00', '2020-02-01 00:00:00', '2020-04-01 00:00:00']],
    "spend": [1000.0, 1000.0, 1000.0],
}).set_index("cohort_start_at")
EXERCISE_SPEND_DF

,spend
cohort_start_at,
2020-01-01,1000.0
2020-02-01,1000.0
2020-04-01,1000.0


# Exercise 2

In [65]:
 def apply_predictions_list(m0 : float, churn : float, actual_values : list = list[float], n_months : int =
       6) -> list:
           """
           Extend a list of actual values to n_months length with predictions.

           If no actual values exist, start with m0.
           Each predicted value = previous value * (1 - churn)
           """
           # Start with a copy of actual values, or empty list if none provided
           result = actual_values.copy() if actual_values else []

           # If no values present, use m0 as the first value (don't round yet)
           if len(result) == 0:
               result.append(m0)

           # Extend with predictions until we reach n_months (don't round yet)
           while len(result) < n_months:
               # Each predicted value is the previous value multiplied by (1 - churn)
               next_value = result[-1] * (1 - churn)
               result.append(next_value)

           # Round everything at the end
           result = [round(value, 2) for value in result]

           return result

In [75]:
def apply_predictions_to_cohort_df(cohort_df: pd.DataFrame, spend_df: pd.DataFrame, predictions_dict: dict,
       n_months: int = 12) -> pd.DataFrame:
           """
           Apply predictions to extend each cohort's actual values.

           For each cohort (row), get actual values and extend with predictions
           based on m0 and churn from predictions_dict.

           m0 is a percentage that should be multiplied by the spend amount to get the actual dollar value.
           """
           m0_percent = predictions_dict["m0"]
           churn = predictions_dict["churn"]

           # Create result dataframe with column indices from 0 to n_months-1
           result_df = pd.DataFrame(index=cohort_df.index, columns=range(n_months))

           # Process each cohort
           for cohort_date in cohort_df.index:
               # Get actual values for this cohort (drop NaN values)
               actual_values = cohort_df.loc[cohort_date].dropna().tolist()

               # Get the spend amount for this cohort to calculate m0 in dollars
               spend_amount = spend_df.loc[cohort_date, 'spend']
               m0_dollars = m0_percent * spend_amount

               # Apply predictions to extend to n_months
               predicted_values = apply_predictions_list(m0_dollars, churn, actual_values, n_months)

               # Update the row with predicted values
               for i, value in enumerate(predicted_values):
                   result_df.loc[cohort_date, i] = value

           return result_df


In [76]:
# We will extend the cohort dataframe from exercise 1
EXERCISE_2_BEST_PREDICTIONS_DICT = {
        "m0": 0.4,
        "churn": 0.1,
}
# Showing how predictions look different per scenario
# EXERCISE_2_AVERAGE_PREDICTIONS_DICT = {
#         "m0": 0.3,
#         "churn": 0.2,
# }
# EXERCISE_2_WORST_PREDICTIONS_DICT = {
#         "m0": 0.1,
#         "churn": 0.3,
# }

In [77]:
# Compare your results to these expected values
"""
                0      1      2      3       4       5
2020-01-01  200.0  100.0   90.0   81.0   72.90   65.61
2020-02-01  400.0  300.0  270.0  243.0  218.70  196.83
2020-04-01  500.0  450.0  405.0  364.5  328.05  295.25
"""
EXPECTED_PREDICTED_SERIES = [0.4, 0.36, 0.32, 0.29, 0.26, 0.24, 0.21, 0.19, 0.17, 0.15, 0.14, 0.13]
assert apply_predictions_list(EXERCISE_2_BEST_PREDICTIONS_DICT["m0"], EXERCISE_2_BEST_PREDICTIONS_DICT["churn"], [], 12) == EXPECTED_PREDICTED_SERIES


apply_predictions_to_cohort_df(EXERCISE_1_COHORT_DF, EXERCISE_SPEND_DF, EXERCISE_2_BEST_PREDICTIONS_DICT)

,0,1,2,3,4,5,6,7,8,9,10,11
2020-01-01,200.0,100.0,90.0,81.0,72.9,65.61,59.05,53.14,47.83,43.05,38.74,34.87
2020-02-01,400.0,300.0,270.0,243.0,218.7,196.83,177.15,159.43,143.49,129.14,116.23,104.6
2020-04-01,500.0,450.0,405.0,364.5,328.05,295.25,265.72,239.15,215.23,193.71,174.34,156.91


# Exercise 3

In [78]:
def apply_threshold_to_cohort_df(cohort_df : pd.DataFrame, spend_df : pd.DataFrame, threshold : list[dict]) -> pd.DataFrame:
    """
    Check if each cohort's payments meet the minimum thresholds.
    
    Returns a boolean dataframe where:
    - True = threshold met (payment >= spend * minimum_payment_percent)
    - False = threshold breached (payment < threshold or NaN)
    - True if no threshold defined for that payment period (fail open)
    """
    # Convert threshold list to dict for faster lookup: month -> min_percent
    threshold_dict = {t["payment_period_month"]: t["minimum_payment_percent"] for t in threshold}
    
    # Create result dataframe with same shape as cohort_df, default to True (fail open)
    result_df = pd.DataFrame(True, index=cohort_df.index, columns=cohort_df.columns)
    
    # Check each payment period that has a threshold defined
    for period_month, min_percent in threshold_dict.items():
        if period_month in cohort_df.columns:
            # Calculate minimum required payment for each cohort: spend * min_percent
            min_payments = spend_df['spend'] * min_percent
            
            # Get actual payments for this period
            payments = cohort_df[period_month]
            
            # Check if payment >= minimum (NaN comparisons return False automatically)
            result_df[period_month] = payments >= min_payments
    
    return result_df

In [79]:
EXAMPLE_THRESHOLD_DICT = [{
    "payment_period_month": 1,
    "minimum_payment_percent": 0.25
}]

In [80]:
""" The above should return a threshold_df that looks like this
                0     1
2020-01-01   True  False
2020-02-01   True  True
2020-04-01   True  False
"""
threshold_df = apply_threshold_to_cohort_df(EXERCISE_1_COHORT_DF, EXERCISE_SPEND_DF, EXAMPLE_THRESHOLD_DICT)
threshold_df


,0,1
2020-01-01,True,False
2020-02-01,True,True
2020-04-01,True,False


# Exercise 4

In [81]:
def get_cvf_cashflows_df(cohort_df: pd.DataFrame, spend_df: pd.DataFrame, threshold: list[dict], trade_list: list[dict]) -> pd.DataFrame:
    """
    Calculate CVF cashflows for each cohort based on trades, thresholds, and caps.
    
    Steps:
    1. Check thresholds to determine where to use 100% vs sharing_percentage
    2. Apply sharing percentages and caps from trade_list
    3. Convert payment periods to calendar dates
    4. Zero out payments after 12 months or before cohort starts
    
    Note: Pass in a predicted cohort_df (via apply_predictions_to_cohort_df) if you want predictions.
    """
    # Step 1: Get threshold mask (True = threshold met, False = breached)
    threshold_mask = apply_threshold_to_cohort_df(cohort_df, spend_df, threshold)
    
    # Step 2: Create trade lookup dict by cohort_start
    trade_dict = {trade["cohort_start"]: trade for trade in trade_list}
    
    # Step 3: Calculate cashflows for each cohort
    cashflow_df = pd.DataFrame(0.0, index=cohort_df.index, columns=cohort_df.columns)
    
    for cohort_date in cohort_df.index:
        trade = trade_dict.get(cohort_date)
        if not trade:
            continue
            
        sharing_pct = trade["sharing_percentage"]
        cash_cap = trade["cash_cap"]
        cumulative = 0.0
        
        for period in cohort_df.columns:
            if period not in cohort_df.columns:
                continue
                
            payment = cohort_df.loc[cohort_date, period]
            
            # Handle NaN payments
            if pd.isna(payment):
                cashflow_df.loc[cohort_date, period] = 0.0
                continue
            
            # Determine sharing percentage: 100% if threshold breached, else trade sharing_pct
            threshold_met = threshold_mask.loc[cohort_date, period]
            effective_sharing = sharing_pct if threshold_met else 1.0
            
            # Calculate cashflow
            cashflow = payment * effective_sharing
            
            # Apply cash cap
            if cumulative + cashflow > cash_cap:
                cashflow = cash_cap - cumulative
                cashflow_df.loc[cohort_date, period] = round(cashflow, 2)
                cumulative = cash_cap
                # Stop collecting after cap is reached
                break
            else:
                cashflow_df.loc[cohort_date, period] = round(cashflow, 2)
                cumulative += cashflow
    
    # Step 4: Convert columns from payment periods to calendar dates
    # Find the date range we need to cover
    max_date = cohort_df.index.max() + pd.DateOffset(months=11)
    date_range = pd.date_range(start=cohort_df.index.min(), end=max_date, freq='MS')
    
    # Create result dataframe with calendar dates as columns
    result_df = pd.DataFrame(0.0, index=cohort_df.index, columns=date_range)
    
    for cohort_date in cohort_df.index:
        for period in cashflow_df.columns:
            # Convert period to actual calendar date
            calendar_date = cohort_date + pd.DateOffset(months=int(period))
            
            # Only include if within 12 months and calendar_date exists in result columns
            if period < 12 and calendar_date in result_df.columns:
                result_df.loc[cohort_date, calendar_date] = cashflow_df.loc[cohort_date, period]
    
    return result_df

In [82]:
"""This represents the final output of the function when run only on the cohort_df.

            2020-01-01  2020-02-01  2020-03-01  2020-04-01  2020-05-01
# We have not reached the cash_cap, but we are collecting 100% payments in month 1
2020-01-01       160.0       100.0         0.0         0.0         0.0
# We have reached the cash_cap
2020-02-01         0.0       320.0        80.0         0.0         0.0
# We have not reached the cash_cap
2020-04-01         0.0         0.0         0.0       250.0         0.0
"""
EXAMPLE_TRADE_LIST_1 = [{
    "cohort_start": pd.Timestamp('2020-01-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 500,
},
{
    "cohort_start": pd.Timestamp('2020-02-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 400,
},
{
    "cohort_start": pd.Timestamp('2020-04-01 00:00:00'),
    "sharing_percentage": 0.5,
    "cash_cap": 500,
}]



cvf_cashflows_df = get_cvf_cashflows_df(EXERCISE_1_COHORT_DF, EXERCISE_SPEND_DF, EXAMPLE_THRESHOLD_DICT, EXAMPLE_TRADE_LIST_1)
cvf_cashflows_df


,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01
2020-01-01,160.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-01,0.0,320.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-04-01,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:

"""This represents the final output of the function when run on the best predicted cohort_df

            2020-01-01  2020-02-01  2020-03-01  2020-04-01  2020-05-01  \
# We do not hit our cash cap, even after collecting 100% payments in month 0
2020-01-01       200.0        80.0        90.0        64.8       58.32
# We hit our cash cap, after collecting 100% payments in month 2
2020-02-01         0.0       320.0       240.0       270.0      194.40
# We hit our cash cap, without collection any payments at 100%
2020-04-01         0.0         0.0         0.0       250.0      225.00

            2020-06-01  2020-07-01  2020-08-01  2020-09-01  2020-10-01  \
2020-01-01       52.49       47.24       42.51       38.26       34.44
2020-02-01       75.60        0.00        0.00        0.00        0.00
2020-04-01      202.50      182.25      164.02       76.23        0.00

            2020-11-01  2020-12-01  2021-01-01  2021-02-01  2021-03-01
2020-01-01       30.99        27.9         0.0         0.0         0.0
2020-02-01        0.00         0.0         0.0         0.0         0.0
2020-04-01        0.00         0.0         0.0         0.0         0.0
"""
EXAMPLE_TRADE_LIST_2 = [{
    "cohort_start": pd.Timestamp('2020-01-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 1100,
},
{
    "cohort_start": pd.Timestamp('2020-02-01 00:00:00'),
    "sharing_percentage": 0.8,
    "cash_cap": 1100,
},
{
    "cohort_start": pd.Timestamp('2020-04-01 00:00:00'),
    "sharing_percentage": 0.5,
    "cash_cap": 1100,
}]
EXAMPLE_THRESHOLD_DICT_2 = [
    {
        "payment_period_month": 0,
        "minimum_payment_percent": 0.3
    },
    {
        "payment_period_month": 2,
        "minimum_payment_percent": 0.3
    },
]


predicted_cohort_df = apply_predictions_to_cohort_df(EXERCISE_1_COHORT_DF, EXERCISE_SPEND_DF, EXERCISE_2_BEST_PREDICTIONS_DICT)
predicted_cvf_cashflows_df = get_cvf_cashflows_df(predicted_cohort_df, EXERCISE_SPEND_DF, EXAMPLE_THRESHOLD_DICT_2, EXAMPLE_TRADE_LIST_2)
predicted_cvf_cashflows_df

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01
2020-01-01,200.0,80.0,90.0,64.8,58.32,52.49,47.24,42.51,38.26,34.44,30.99,27.9,0.0,0.0,0.0
2020-02-01,0.0,320.0,240.0,270.0,194.40,75.60,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
2020-04-01,0.0,0.0,0.0,250.0,225.00,202.50,182.25,164.03,76.23,0.00,0.00,0.0,0.0,0.0,0.0
